<a href="https://colab.research.google.com/github/MecksonMfema/ws-eng-conduit-ai-assessment/blob/master/Solve_Business_Problems_with_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solve Business Problems with AI

## Objective
Develop a proof-of-concept application to intelligently process email order requests and customer inquiries for a fashion store. The system should accurately categorize emails as either product inquiries or order requests and generate appropriate responses using the product catalog information and current stock status.

You are encouraged to use AI assistants (like ChatGPT or Claude) and any IDE of your choice to develop your solution. Many modern IDEs (such as PyCharm, or Cursor) can work with Jupiter files directly.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

### Instructions

- Implement all requirements using advanced Large Language Models (LLMs) to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Use Retrieval-Augmented Generation (RAG) and vector store techniques where applicable to retrieve relevant information and generate responses.
- You are provided with a temporary OpenAI API key granting access to GPT-4o, which has a token quota. Use it wisely or use your own key if preferred.
- Address the requirements in the order listed. Review them in advance to develop a general implementation plan before starting.
- Your deliverables should include:
   - Code developed within this notebook.
   - A single spreadsheet containing results, organized across separate sheets.
   - Comments detailing your thought process.
- You may use additional libraries (e.g., langchain) to streamline the solution. Use libraries appropriately to align with best practices for AI and LLM tools.
- Use the most suitable AI techniques for each task. Note that solving tasks with traditional programming methods will not earn points, as this assessment evaluates your knowledge of LLM tools and best practices.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process orders
  - For each order request, verify product availability in stock.
  - If the order can be fulfilled, create a new order line with the status “created”.
  - If the order cannot be fulfilled due to insufficient stock, create a line with the status “out of stock” and include the requested quantity.
  - Update stock levels after processing each order.
  - Record each product request from the email.
  - **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate responses
  - Create response emails based on the order processing results:
      - If the order is fully processed, inform the customer and provide product details.
      - If the order cannot be fulfilled or is only partially fulfilled, explain the situation, specify the out-of-stock items, and suggest alternatives or options (e.g., waiting for restock).
  - Ensure the email tone is professional and production-ready.
  - **Output**: Populate the **order-response** sheet with columns: email ID, response.

#### 3. Handle product inquiry

Customers may ask general open questions.
  - Respond to product inquiries using relevant information from the product catalog.
  - Ensure your solution scales to handle a full catalog of over 100,000 products without exceeding token limits. Avoid including the entire catalog in the prompt.
  - **Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria
- **Advanced AI Techniques**: The system should use Retrieval-Augmented Generation (RAG) and vector store techniques to retrieve relevant information from data sources and use it to respond to customer inquiries.
- **Tone Adaptation**: The AI should adapt its tone appropriately based on the context of the customer's inquiry. Responses should be informative and enhance the customer experience.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational as described.
- **Code Quality and Clarity**: The code should be well-organized, with clear logic and a structured approach. It should be easy to understand and maintain.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the output spreadsheet. Ensure the format of each output matches the requirements—do not add extra columns or sheets.
- **Accuracy of Outputs**: The accuracy of the generated outputs is crucial and will significantly impact the evaluation of your submission.

We look forward to seeing your solution and your approach to solving real-world problems with AI technologies.

# Prerequisites

### Configure OpenAI API Key.

In [2]:
# Install the OpenAI Python package.
%pip install openai httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.27.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.
firebase-admin 6.9.0 requires httpx[http2]==0.28.1, but you have httpx 0.27.2 which is incompatible.


**IMPORTANT: If you are going to use our custom API Key then make sure that you also use custom base URL as in example below. Otherwise it will not work.**

In [3]:
# Code example of OpenAI communication

from openai import OpenAI

client = OpenAI(
    # In order to use provided API key, make sure that models you create point to this custom base URL.
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    # The temporary API key giving access to ChatGPT 4o model. Quotas apply: you have 500'000 input and 500'000 output tokens, use them wisely ;)
    api_key='a0BIj000002xWtaMAE'
)

In [4]:
# Code example of reading input data
%load_ext cudf.pandas
import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

/usr/local/lib/python3.11/dist-packages/cudf/utils/gpu_utils.py:75: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))
/usr/local/lib/python3.11/dist-packages/cudf/pandas/__init__.py:64: UserWarning: Function "cuInit" not found
  warnings.warn(str(e))


,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


In [5]:
# Code example of generating output document

# Creates a new shared Google Worksheet every invocation with the proper structure
# Note: This code should be executed from the google colab once you are ready, it will not work locally
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe

# IMPORTANT: You need to authenticate the user to be able to create new worksheet
# Insert the authentication snippet from the official documentation to create a google client:
# https://colab.research.google.com/notebooks/io.ipynb#scrollTo=qzi9VsEqzI-o

# This code goes after creating google client
output_document = gc.create('Solving Business Problems with AI - Output')

# Create 'email-classification' sheet
email_classification_sheet = output_document.add_worksheet(title="email-classification", rows=50, cols=2)
email_classification_sheet.update([['email ID', 'category']], 'A1:B1')

# Example of writing the data into the sheet
# Assuming you have your classification in the email_classification_df DataFrame
# set_with_dataframe(email_classification_sheet, email_classification_df)
# Or directly update cells: https://docs.gspread.org/en/latest/user-guide.html#updating-cells

# Create 'order-status' sheet
order_status_sheet = output_document.add_worksheet(title="order-status", rows=50, cols=4)
order_status_sheet.update([['email ID', 'product ID', 'quantity', 'status']], 'A1:D1')

# Create 'order-response' sheet
order_response_sheet = output_document.add_worksheet(title="order-response", rows=50, cols=2)
order_response_sheet.update([['email ID', 'response']], 'A1:B1')

# Create 'inquiry-response' sheet
inquiry_response_sheet = output_document.add_worksheet(title="inquiry-response", rows=50, cols=2)
inquiry_response_sheet.update([['email ID', 'response']], 'A1:B1')

# Share the spreadsheet publicly
output_document.share('', perm_type='anyone', role='reader')

# This is the solution output link, paste it into the submission form
print(f"Shareable link: https://docs.google.com/spreadsheets/d/{output_document.id}")

NameError: name 'gc' is not defined

# Task 1. Classify emails

In [6]:
import re
import numpy as np

In [7]:
# Preprocessing
def clean_text(text):
    if pd.isna(text):
        return ""
    text = re.sub(r'\[.*?\]', '', text)  # Remove product codes
    text = re.sub(r'\b[A-Z]{3}\d{4}\b', '', text)  # Remove product IDs
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

emails_df['clean_subject'] = emails_df['subject'].apply(clean_text)
emails_df['clean_body'] = emails_df['message'].apply(clean_text)

In [8]:

class ProductVectorStore:
  def __init__(self, products_df):
    self.products = products_df
    self.embeddings = {}
    self.contexts = []

  def generate_embeddings(self):
      for _, row in self.products.iterrows():
        context = (
          f"Product: {row['name']} | "
          f"Category: {row['category']} | "
          f"Description: {row['description']} | "
          f"Stock: {row['stock']} | "
          f"Season: {row['seasons']}"
        )
        self.contexts.append(context)

      response = client.embeddings.create(
        input=self.contexts,
        model="text-embedding-3-small"
      )
      self.embeddings = {ctx: emb.embedding for ctx, emb in zip(self.contexts, response.data)}

  def retrieve_relevant(self, query, top_k=2):
    query_emb = client.embeddings.create(
      input=[query],
      model="text-embedding-3-small"
    ).data[0].embedding

    similarities = []
    for text, emb in self.embeddings.items():
      cos_sim = np.dot(emb, query_emb)
      similarities.append((text, cos_sim))

    return sorted(similarities, key=lambda x: x[1], reverse=True)[:top_k]

In [13]:
class EnhancedEmailClassifier:
    INQUIRY_KEYWORDS = [
        'question', 'inquiry', 'recommend', 'advice',
        'information', 'details', 'compare', 'difference',
        'suitable', 'appropriate', 'help', 'considering',
        'thinking about', 'wondering', 'curious', 'confirm'
    ]

    ORDER_KEYWORDS = [
        'order', 'buy', 'purchase', 'send me',
        'deliver', 'quantity', 'how many', 'take all',
        'remaining stock', 'place order', 'commit to',
        'immediately', 'right now', 'today', 'asap'
    ]

    # FUTURE purchase patterns
    FUTURE_PATTERNS = [
        'will', 'later', 'future', 'next month',
        'plan to', 'looking to buy', 'need to buy'
    ]

    # Strong order regex patterns
    STRONG_ORDER_PHRASES = [
        r'\bsend me\b', r'\btake all\b', r'\bremaining stock\b',
        r'\bplace.*order\b', r'\bcommit to\b',
        r"\bi would like to (?:buy|order)\b", r"\bi'd like to (?:buy|order)\b",
        r"\bcould you send me\b", r"\bcan you send me\b",
        r"\bplease send me\b", r"\bplease ship\b"
    ]

    def __init__(self, vector_store):
        self.vector_store = vector_store
        self.cache = {}

    def classify_email(self, email_id: str, subject: str, body: str) -> str:
        key = (subject or '')[:30] + (body or '')[:30]
        if key in self.cache:
            return self.cache[key]

        pre = self.rule_based_classifier(subject, body)
        llm = self.llm_classification(subject, body, pre)
        final = self.validate_classification(llm, subject, body)

        self.cache[key] = final
        return final

    def rule_based_classifier(self, subject: str, body: str) -> str:
        text = f"{subject or ''} {body or ''}".lower()

        # 1) Strong order indicators
        for pattern in self.STRONG_ORDER_PHRASES:
            if re.search(pattern, text):
                return "order request"

        # 2) Future intent becomes inquiry only if no order keyword
        has_future = any(fp in text for fp in self.FUTURE_PATTERNS)
        has_order_kw = any(re.search(rf"\b{kw}\b", text) for kw in self.ORDER_KEYWORDS)
        if has_future and not has_order_kw:
            return "product inquiry"

        # 3) Strong inquiry indicators
        if any(re.search(rf"\b{kw}\b", text) for kw in self.INQUIRY_KEYWORDS):
            return "product inquiry"

        # 4) Question mark
        if '?' in text:
            return "product inquiry"

        return None

    def llm_classification(self, subject, body, pre_class):
        # If rule-based is confident, use it
        if pre_class:
            return pre_class

        email_content = f"Subject: {subject}\nBody: {body}"

        # Get relevant context
        context_items = self.vector_store.retrieve_relevant(email_content)
        context_str = "\n".join([item[0] for item in context_items])

        # Enhanced system prompt
        system_prompt = """
        You are an email classifier for a fashion retailer. Strictly categorize emails as:
        - "product inquiry": Questions about products, features, availability, colors, sizes, recommendations, or future purchase considerations
        - "order request": Explicit, immediate purchase intent with specific products and quantities

        Key differences:
        - "I want to buy" = ORDER if immediate action is stated
        - "I'm thinking of buying" = INQUIRY
        - "How much is" = INQUIRY
        - "Send me 2 of these" = ORDER

        Respond ONLY with "product inquiry" or "order request"
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"Relevant Products:\n{context_str}\n\nEmail:\n{email_content}"}
                ],
                temperature=0.0,  # More deterministic
                max_tokens=10
            )
            return response.choices[0].message.content.strip().lower()
        except Exception:
            return self.fallback_classifier(subject, body)

    def fallback_classifier(self, subject: str, body: str) -> str:
        text = f"{subject or ''} {body or ''}".lower()
        inq_score = sum(bool(re.search(rf"\b{kw}\b", text)) for kw in self.INQUIRY_KEYWORDS)
        ord_score = sum(bool(re.search(rf"\b{kw}\b", text)) for kw in self.ORDER_KEYWORDS)
        future = any(fp in text for fp in self.FUTURE_PATTERNS)
        if ord_score > inq_score and not future:
            return "order request"
        return "product inquiry"

    def validate_classification(self, classification: str, subject: str, body: str) -> str:
        text = f"{subject or ''} {body or ''}".lower()
        future = any(fp in text for fp in self.FUTURE_PATTERNS)
        valid = classification if classification in ["product inquiry", "order request"] else "product inquiry"
        if valid == "order request" and future:
            return "product inquiry"
        return valid



In [14]:
# --- Execute Classification ---
# Initialize vector store
vector_store = ProductVectorStore(products_df)
vector_store.generate_embeddings()

# Initialize classifier
classifier = EnhancedEmailClassifier(vector_store)

# Classify emails
results = []
for _, row in emails_df.iterrows():
  classification = classifier.classify_email(
    row['email_id'],
    row['clean_subject'],
    row['clean_body']
  )
  results.append({
    'email ID': row['email_id'],
    'category': classification
  })

# Create output DataFrame
classification_df = pd.DataFrame(results)

# Save results
classification_df.to_csv('improved_email_classification.csv', index=False)
print("Improved classification completed!")

# Display results
print(classification_df)

Improved classification completed!
   email ID         category
0      E001    order request
1      E002  product inquiry
2      E003  product inquiry
3      E004    order request
4      E005  product inquiry
5      E006    order request
6      E007    order request
7      E008    order request
8      E009  product inquiry
9      E010    order request
10     E011  product inquiry
11     E012  product inquiry
12     E013  product inquiry
13     E014    order request
14     E015  product inquiry
15     E016  product inquiry
16     E017    order request
17     E018    order request
18     E019    order request
19     E020  product inquiry
20     E021  product inquiry
21     E022    order request
22     E023    order request


In [15]:
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe

auth.authenticate_user()  # Prompt user to authenticate in Colab
creds, _ = default()      # Get default credentials
gc = gspread.authorize(creds)

# 2. Create the output Google Spreadsheet
doc_title = 'Solving Business Problems with AI - Output'
output_document = gc.create(doc_title)
print(f"Created spreadsheet: {doc_title} (ID: {output_document.id})")

# 3. Share the spreadsheet publicly (read-only)
# Replace '' with a specific email if needed
auth_role = 'reader'
auth_type = 'anyone'
output_document.share('', perm_type=auth_type, role=auth_role)
print("Spreadsheet shared publicly with read-only access.")

Created spreadsheet: Solving Business Problems with AI - Output (ID: 1zOkSYLLu2Ng5PcJRnInkDBa8wa4RewBRgvPmEzcFhXQ)
Spreadsheet shared publicly with read-only access.


In [16]:
def create_and_fill(title, df, headers):
    """
    Creates a worksheet named `title` and writes `headers` then `df`.
    """
    worksheet = output_document.add_worksheet(title=title, rows=str(len(df)+10), cols=str(len(df.columns)+1))
    # Write header row
    worksheet.update([headers], 'A1')
    # Write DataFrame
    set_with_dataframe(worksheet, df, include_index=False, include_column_header=False, row=2, col=1)
    print(f"Populated '{title}' with {len(df)} rows.")

In [17]:
create_and_fill('email-classification', classification_df, ['email ID', 'category'])

Populated 'email-classification' with 23 rows.


In [18]:
print(f"Shareable link: https://docs.google.com/spreadsheets/d/{output_document.id}")

Shareable link: https://docs.google.com/spreadsheets/d/1zOkSYLLu2Ng5PcJRnInkDBa8wa4RewBRgvPmEzcFhXQ


# Task 2. Process order requests

# Task 3. Handle product inquiry